In [ ]:
import time
import pandas as pd
import numpy as np
import datetime as dt
from datetime import date, timedelta
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.max_colwidth', None)

In [ ]:
s = 2020
players = pd.read_csv('../data/database-players.csv')
nfl_weeks = pd.read_csv('../data/NFL-Week-Dates.csv')
nfl_weeks['Week'] = nfl_weeks['Week'].astype(str)
def pre_thu(d):
    days_behind = 3 - d.weekday()
    if days_behind > 0:
        days_behind -= 7
    return d + dt.timedelta(days_behind)
def findnth(haystack, needle, n):
    parts= haystack.split(needle, n+1)
    if len(parts)<=n+1:
        return -1
    return len(haystack)-len(parts[-1])-len(needle)
def renaming_cols(col):
    if "Unnamed:" in col:
        new = col[findnth(col, "'", 2)+1:-2]
    elif "Passing" in col or "Rushing" in col or "Receiving" in col or "Fumbles" in col or "Scoring" in col or "Tackles" in col or "Punting" in col:
        new = col[findnth(col, "'", 0)+1:findnth(col, "'", 1)] + "_" + col[findnth(col, "'", 2)+1:findnth(col, "'", 3)]
    elif "Punt Returns" in col or "ST Snaps" in col or "Kick Returns" in col:
        new = col[findnth(col, "'", 0)+1:findnth(col, " ", 0)] + "_" + col[findnth(col, " ", 0)+1:findnth(col, "'", 1)] + "_" + col[findnth(col, "'", 2)+1:findnth(col, "'", 3)]
    elif "Def. Snaps" in col or "Off. Snaps" in col:
        new = col[findnth(col, "'", 0)+1:findnth(col, " ", 0)-1] + "_" + col[findnth(col, " ", 0)+1:findnth(col, "'", 1)] + "_" + col[findnth(col, "'", 2)+1:findnth(col, "'", 3)]
    elif "Def Interceptions" in col:
        new = col[findnth(col, "'", 0)+1:findnth(col, " ", 0)] + "_" + col[findnth(col, " ", 0)+1:findnth(col, " ", 0)+4] + "_" + col[findnth(col, "'", 2)+1:findnth(col, "'", 3)]
    else:
        new = col
    return new

In [ ]:
# player stats clean
start = time.time()
stats = pd.read_csv(f'../data/raw-data/player-weekly-stats-{s}-raw.csv', low_memory=False)
stats = stats.rename(columns=renaming_cols)
stats = stats.rename(columns = {"Unnamed: 6_level_1": "Home_Away", "Tm": "Team", "Passing_Yds.1": "Passing_Sk_Yds", "GS": "Gm_Start", "G#": "Gm_Num"})
stats = pd.merge(left=stats, 
                 right=players, 
                 how='outer', 
                 left_on='Player_Address', 
                 right_on='Address')
stats = stats.groupby(stats.columns, axis=1).sum()
stats.drop(['First_Year', 'Last_Year', 'Rk', 'Address'], axis=1, inplace=True)
stats.replace({'Home_Away': {'@': 'Away', 0: 'Home'}, 
               'Gm_Start': {'*': True, None: False}}, inplace=True)
stats.dropna(subset=['Player_Address'], inplace=True)
stats[['Off_Snaps_Pct', 
       'Def_Snaps_Pct', 
       'ST_Snaps_Pct',
       'Player_Address']] = stats[['Off_Snaps_Pct', 
                                   'Def_Snaps_Pct', 
                                   'ST_Snaps_Pct', 
                                   'Player_Address']].astype(str)
stats['Off_Snaps_Pct'] = list(map(lambda x: x[:-1], stats['Off_Snaps_Pct'].values))
stats['Def_Snaps_Pct'] = list(map(lambda x: x[:-1], stats['Def_Snaps_Pct'].values))
stats['ST_Snaps_Pct'] = list(map(lambda x: x[:-1], stats['ST_Snaps_Pct'].values))
stats['Player_Address'] = list(map(lambda x: x[11:], stats['Player_Address'].values))
stats = stats[['Player_Address', 'Player', 'Position', 'Age', 
               'Team', 'Home_Away', 'Opp', 'Result', 'Week', 
               'Gm_Num', 'Season', 'Gm_Start', 'Age', 'Date', 'Off_Snaps_Num',
               'Off_Snaps_Pct', 'Def_Snaps_Num', 'Def_Snaps_Pct',
               'ST_Snaps_Num', 'ST_Snaps_Pct', 'Passing_Att',  'Passing_Cmp',
               'Passing_Cmp%', 'Passing_Yds', 'Passing_TD', 'Passing_Int',
               'Passing_Rate', 'Passing_Sk', 'Passing_Sk_Yds', 'Passing_Y/A',
               'Passing_AY/A', 'Rushing_Att', 'Rushing_Yds', 'Rushing_Y/A',
               'Rushing_TD', 'Receiving_Tgt', 'Receiving_Rec', 'Receiving_Yds', 'Receiving_TD',
               'Receiving_Y/R', 'Receiving_Ctch%', 'Receiving_Y/Tgt', 'Kick_Returns_Rt',
               'Kick_Returns_Yds', 'Kick_Returns_Y/Rt', 'Kick_Returns_TD', 'Punt_Returns_Ret', 'Punt_Returns_Yds',
               'Punt_Returns_TD', 'Punt_Returns_Y/R', 'Punting_Pnt', 'Punting_Yds', 'Punting_Y/P',
               'Punting_Blck', 'Scoring_TD', 'Scoring_Pts', 'Scoring_XPM',
               'Scoring_XPA', 'Scoring_XP%', 'Scoring_FGM', 'Scoring_FGA',
               'Scoring_FG%', 'Scoring_2PM', 'Scoring_Sfty', 'Sk',
               'Tackles_Solo', 'Tackles_Ast', 'Tackles_Comb', 'Tackles_TFL',
               'Tackles_QBHits', 'Def_Int_Int', 'Def_Int_Yds', 'Def_Int_TD',
               'Def_Int_PD', 'Fumbles_Fmb', 'Fumbles_FL', 'Fumbles_FF',
               'Fumbles_FR', 'Fumbles_Yds', 'Fumbles_TD']]

In [ ]:
# player injurys clean
injury = pd.read_csv(f'../data/raw-data/nfl-injury-report-{s}-raw.csv', low_memory=False)
injury = pd.melt(injury,
                 id_vars=['Player', 'Team', 'Season', 'Player_Address'], 
                 var_name='Date', 
                 value_name='Status')
injury[['Date','Opp']] = injury.Date.str.split('vs. ', expand=True)
injury[['Month','Day']] = injury.Date.str.split('/', expand=True)
injury[['Status','Injury']] = injury.Status.str.split(":", expand=True)
injury.dropna(axis=0, subset=['Status','Injury'], how='all', inplace=True)
injury[['Season', 'Month', 'Day']] = injury[['Season', 'Month', 'Day']].astype(int)
injury['Date'] = injury['Date'] + '/' + (np.where(injury['Month'] <= 2, injury['Season'] + 1, injury['Season'])).astype(str)
injury['Date'] = pd.to_datetime(injury['Date'])
injury['week_start_nfl'] = injury['Date'].apply(pre_thu)
nfl_weeks['Start Date'] = pd.to_datetime(nfl_weeks['Start Date'])
injury = pd.merge(left=injury, 
                  right=nfl_weeks, 
                  how='left', 
                  left_on='week_start_nfl', 
                  right_on='Start Date')
injury.replace({'Team':
                   {'crd': 'ARI', 'atl': 'ATL', 'rav': 'BAL', 'buf': 'BUF', 
                    'car': 'CAR', 'chi': 'CHI', 'cin': 'CIN', 'cle': 'CLE', 
                    'dal': 'DAL', 'den': 'DEN', 'det': 'DET', 'gnb': 'GNB',
                    'htx': 'HOU', 'clt': 'IND', 'jax': 'JAX', 'kan': 'KAN', 
                    'sdg': 'LAC', 'ram': 'LAR', 'mia': 'MIA', 'min': 'MIN', 
                    'nor': 'NOR', 'nwe': 'NWE', 'nyg': 'NYG', 'nyj': 'NYJ', 
                    'rai': 'OAK', 'phi': 'PHI', 'pit': 'PIT', 'sea': 'SEA',
                    'sfo': 'SFO', 'tam': 'TAM',                    'oti':'TEN','was':'WAS'}},
               inplace=True)
injury['Injury'] = injury['Injury'].str.strip(' ')
injury.replace({'Injury': {'right':'','left':'','Right':'','Left':'','Biceps':'Bicep',
                          'Triceps':'Tricep','Ankles':'Ankle','hip':'Hip','Hips':'Hip','Knees':'Knee',
                          'Virus':'Illness','Triceps':'Tricep','Oblique':'Abdomen',
                          'NotInjuryRelated':'Not Injury Related','MedicalIllness':'Illness',
                          'LowerLeg':'Lower Leg','CoreMuscle':'Abdomen','Abdominal':'Abdomen'}},
               regex=True, 
               inplace=True)
injury['Player'] = injury['Player'] + ' '
injury.replace({'Player': {' Jr. ': '', ' Jr ': '', ' Sr. ': '', ' Sr ': '', ' III ': '', ' II ': '', ' IV ': '', ' V ': ''}}, 
               regex=True, 
               inplace=True)
injury['Player'] = injury['Player'].str.strip(' ')
injury['Player'] = injury['Player'].str.replace('.', '')
injury.drop(['Month', 'Day', 'week_start_nfl', 'Start Date'], axis=1, inplace=True)
injury.dropna(subset=['Week'], inplace=True)
injury[['Player', 'Week', 'Season']] = injury[['Player', 'Week', 'Season']].astype(str)
injury = injury[['Player', 'Team', 'Opp', 'Date', 'Season', 'Week', 'Status', 'Injury']]
injury[['Specific_Inj', 'Side']] = None
# injury.to_csv(f'../data/clean-data/injury-report-{s}-clean.csv', index=False)